In [ ]:
import matplotlib
import numpy as np
import seaborn as sns
import pandas as pd
import sklearn as sk
import sys
import matplotlib.pyplot as plt
from sklearn import svm
from jedi.api.refactoring import inline

from sklearn.datasets import load_boston
from sklearn.datasets import load_iris, make_moons
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RANSACRegressor
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, roc_curve, roc_auc_score, accuracy_score, classification_report
from sklearn.pipeline import make_pipeline, Pipeline
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.linear_model import Ridge, Lasso, ElasticNet, SGDClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, AdaBoostRegressor, BaggingClassifier, GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_score, cross_val_predict, train_test_split
from sklearn.base import clone
from sklearn.datasets import load_boston
from sklearn.svm import SVR
from sklearn import tree
from sklearn.tree import export_graphviz, DecisionTreeClassifier
import graphviz
from sklearn import preprocessing

import xgboost as xgb

#Combinan múltiples modelos de aprendizaje automático. Pueden ser regresiones o clasificadores
'''

Boostrap Aggregating or Bagging:
1. Bootstrap es muestreo con reemplazo 
2. Combina al promediar el resultado (regresion)
3. Combina al votar (clasificación)
4. Aplica a muchos clasificadores que incluyen Artificial Neural Network (ANN), CART

Pasting
Muestreo sin reemplazo

Boosting 
1. Entrena clasificadores débiles
2. Los añade a un clasificador fuerte final al ponderarlos. (Ponderación por exactitud típicamente)
3. Los datos son reponderados

4. Muestras que fueron mal clasificadas ganan peso 
5. Muestras que fueron bien clasificadas pierden peso 

Stacking
1. También conocido como Generalización apilada (Stacked), o modelo de segundo nivel
2. Combina información de múltiples modelos de predicción para generar un nuevo modelo. Seguido el modelo apilado
    supera cada uno de los modelos individuales debido a su suavizamiento y habilidad de sobresaltar cada modelo base donde se
    desempeña mejor y desacredita aquellos modelos que tienen un pobre rendimiento.
3.  Stacking es el más efectivo cuando los modelos base son muy diferentes
4. Entrenando un algoritmo de aprendizaje para combinar predicciones de otros algoritmos de aprendizaje 
'''

# BAGGING

df = sns.load_dataset('titanic')
print(df.shape)
print(df.head())

#df.dropna(inplace=True)

print(df.head())

print(df['pclass'].unique())

print(df['pclass'].value_counts())

print(df['sex'].value_counts())

df['age'].hist(bins=50)
plt.show()

''' Data Preprocessing '''
subset = df[['pclass', 'sex', 'age', 'survived']].copy()
subset.dropna(inplace=True)

X = subset[['pclass', 'sex', 'age']].copy()

le = preprocessing.LabelEncoder()       # Transforma male/ female en unos y ceros

X['sex'] = le.fit_transform(subset['sex'])

print(X.head())
print(X.shape)
print(X.describe())
print(X.info())

y = subset['survived'].copy()
print(y.value_counts())


# Fit Model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

def print_score(clf, X_train, X_test, y_train, y_test, train=True):
    lb = preprocessing.LabelBinarizer()
    lb.fit(y_train)

    if train:
        '''
        training process
        '''
        res = clf.predict(X_train)
        print("Train Result: \n")
        print("Accuracy score: {0:.4f}\n".format(accuracy_score(y_train, res)))
        print("Classification Report: \n {}\n".format(classification_report(y_train, res)))
        print("Confussion Matrix: \n {}\n".format(confusion_matrix(y_train, res)))
        print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_train), lb.transform(res))))

    elif train == False:

        res_test = clf.predict(X_test)
        print("Train Result: \n")
        print("Accuracy score: {0:.4f}\n".format(accuracy_score(y_test, res_test)))
        print("Classification Report: \n {}\n".format(classification_report(y_test, res_test)))
        print("Confussion Matrix: \n {}\n".format(confusion_matrix(y_test, res_test)))
        print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_test), lb.transform(res_test))))


''' Decision Tree '''

clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

print_score(clf, X_train, X_test, y_train, y_test, train=True)
print('\n *************************************************')
print_score(clf, X_train, X_test, y_train, y_test, train=False)


''' Bagging (oob_score = False) '''

'''
out of bag : Método de medición para la predicción de los errores de bosques aleatorios, arboles de decisión incrementados y otros 
modelos de Machine Learning que utilizan agregación bootstrap (bagging)

'''

bag_clf = BaggingClassifier(base_estimator=clf, n_estimators=1000, bootstrap=True, n_jobs=-1, random_state=42)
bag_clf.fit(X_train, y_train)


print_score(bag_clf, X_train, X_test, y_train, y_test, train=True)
print('\n *************************************************')
print_score(bag_clf, X_train, X_test, y_train, y_test, train=False)


'''
Bagging(oob_score=True)
Utilizamos las muestras out of bag (predicción de errores) para estimar la exactitud generalizada
'''


bag_clf = BaggingClassifier(base_estimator=clf, n_estimators=1000, bootstrap=True, oob_score=True, n_jobs=-1, random_state=42)
bag_clf.fit(X_train, y_train)


print_score(bag_clf, X_train, X_test, y_train, y_test, train=True)
print('\n *************************************************')
print_score(bag_clf, X_train, X_test, y_train, y_test, train=False)


print(bag_clf.oob_score_)




'''
Gradient Boosting Machine

'''

gbc_clf = GradientBoostingClassifier()
gbc_clf.fit(X_train, y_train)

print_score(gbc_clf, X_train, X_test, y_train, y_test, train=True)
print('\n *************************************************')
print_score(gbc_clf, X_train, X_test, y_train, y_test, train=False)


'''
XGBOOST
'''

data = np.random.rand(100,10) # 5 entidades cada una contiene 10 características
label = np.random.randint(2, size=100)
dtrain = xgb.DMatrix(data, label=label)

dtest = dtrain

param = {'bst:max_depth': 2, 'bst:eta' : 1, 'silent': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'auc'

evallist = [(dtest, 'eval'),(dtrain, 'train')]

num_round = 10
bst = xgb.train(param, dtrain, num_round, evallist)